<a href="https://colab.research.google.com/github/JSJeong-me/KOSA-Big-Data_Vision/blob/main/Roboflow_CLIP_Zero_Shot_Cake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to use CLIP Zero-Shot on your own classificaiton dataset

This notebook provides an example of how to benchmark CLIP's zero shot classification performance on your own classification dataset.

[CLIP](https://openai.com/blog/clip/) is a new zero shot image classifier relased by OpenAI that has been trained on 400 million text/image pairs across the web. CLIP uses these learnings to make predicts based on a flexible span of possible classification categories.

CLIP is zero shot, that means **no training is required**. 

Try it out on your own task here!

Be sure to experiment with various text prompts to unlock the richness of CLIP's pretraining procedure.


![Roboflow Wordmark](https://i.imgur.com/dcLNMhV.png)


# Download and Install CLIP Dependencies

In [1]:
#installing some dependencies, CLIP was release in PyTorch
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} -f https://download.pytorch.org/whl/torch_stable.html ftfy regex

import numpy as np
import torch

print("Torch version:", torch.__version__)

CUDA version: 11.0
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1 MB 1.5 MB/s eta 0:03:31tcmalloc: large alloc 1147494400 bytes == 0x55b2e2638000 @  0x7f8086156615 0x55b2a917002c 0x55b2a925017a 0x55b2a9172e4d 0x55b2a9264c0d 0x55b2a91e70d8 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e6f40 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e393b 0x55b2a9265a56 0x55b2a91e2fb3 0x55b2a9265a56 0x55b2a91e2fb3 0x55b2a9265a56 0x55b2a91e2fb3 0x55b2a9174b99 0x55b2a91b7e79 0x55b2a91737b2 0x55b2a91e6e65 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e393b 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e2b0e 0x55b2a917465a 0x55b2a91e2d67 0x55b2a91e1c35
     |█████████████████████████████▏  | 1055.7 MB 1.3 MB/s eta 0:01:18tcmalloc: large alloc 1434370048 bytes == 0x55b326c8e000 @  0x7f8086156615 0x55b2a917002c 0x55b2a925017a 0x55b2a9172e4d 0x55b2a9264c0d 0x55b2a91e70d8 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e6f40 0x55b2a91e1c35 0x55b2a917473a 0x55b2a91e393b 0x55b2a9

In [5]:
!ls -l

total 0


In [6]:
#clone the CLIP repository
!git clone https://github.com/openai/CLIP.git
%cd CLIP

Cloning into 'CLIP'...
remote: Enumerating objects: 142, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 142 (delta 23), reused 33 (delta 15), pack-reused 91
Receiving objects: 100% (142/142), 8.82 MiB | 22.52 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/CLIP


# Download Classification Data or Object Detection Data

We will download the [public flowers classificaiton dataset](https://public.roboflow.com/classification/flowers_classification) from Roboflow. The data will come out as folders broken into train/valid/test splits and seperate folders for each class label.

You can easily download your own dataset from Roboflow in this format, too.

We made a conversion from object detection to CLIP text prompts in Roboflow, too, if you want to try that out.


To get your data into Roboflow, follow the [Getting Started Guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [18]:
!mkdir cake

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ..

/content


In [ ]:
!cp /content/drive/MyDrive/Flowers_Classification.v3-augmented.clip.zip .

In [ ]:
!pwd

/content


In [ ]:
#download classification data
#replace with your link
!curl -L "https://public.roboflow.com/ds/vPLCmk4Knv?key=tCrKLQNpTi" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
!unzip ./Flowers.zip

In [19]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./test/')
class_names.remove('_tokenization.txt')
class_names

['dandelion', 'daisy', 'cake']

In [20]:
class_names

['dandelion', 'daisy', 'cake']

In [10]:
!pwd

/content/CLIP


In [23]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./test/_tokenization.txt

daisy
dandelion
cake

In [22]:
#edit your prompts as you see fit here
%%writefile ./test/_tokenization.txt
daisy
dandelion
cake

Overwriting ./test/_tokenization.txt


In [24]:
candidate_captions = []
with open('./test/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [15]:
!pwd

/content/CLIP


In [14]:
%cd ./CLIP/

[Errno 2] No such file or directory: './CLIP/'
/content/CLIP


# Run CLIP inference on your classification dataset

In [25]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./test/' + cls + '/*.jpg')
    for img in test_imgs:
        print(img)
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            print(pred)
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    print('accuracy on class ' + cls + ' is :' + str(sum(class_correct)/len(class_correct)))
print('accuracy on all is : ' + str(sum(correct)/len(correct)))

./test/dandelion/13916196427_50a611008f_jpg.rf.85e8c90b29e72b6f2eb7b3d321db8f73.jpg
daisy
./test/dandelion/9472854850_fc9e1db673_jpg.rf.56dff30fc81dda636a82985f8a1927d3.jpg
daisy
./test/dandelion/8083321316_f62ea76f72_n_jpg.rf.3e4740907c8bcf183a2afab6e9c25f14.jpg
daisy
./test/dandelion/15358221063_2c6e548e84_jpg.rf.f5be905e876255a3c7d2f760bb63fd48.jpg
daisy
./test/dandelion/34552250422_320900fd8e_n_jpg.rf.a9a6f2855e8c0dc156e4b0877fd0e8d4.jpg
dandelion
./test/dandelion/8797114213_103535743c_m_jpg.rf.132646214f7e9bcbdda402ab688ca868.jpg
daisy
./test/dandelion/13900486390_5a25785645_n_jpg.rf.6ca8083ee0bd606f480c84359984ca19.jpg
daisy
./test/dandelion/1297972485_33266a18d9_jpg.rf.27db91a578818f7158d6d5d98dbfdc8a.jpg
daisy
./test/dandelion/4510938552_6f7bae172a_n_jpg.rf.44e3121f52842ca7b65a91745b892f74.jpg
daisy
./test/dandelion/4489359360_09db62f825_jpg.rf.dd23d2557efa164171ccc721e6f6b275.jpg
daisy
./test/dandelion/14053173516_a00150a919_m_jpg.rf.db62016c6ebd3912dcc994dbe53edaee.jpg
daisy


In [ ]:
#Hope you enjoyed!
#As always, happy inferencing
#Roboflow